<a href="https://colab.research.google.com/github/DAVIDKSOUSA/DATA_SCIENCE/blob/master/An%C3%A1lise_Fundamentalista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Magic Formula

O objetivo desse DataFrame é fazer uma classificação das em empresas segundo a Magic Formula.

In [0]:

import numpy as np
import pandas as pd
import string
import warnings
warnings.filterwarnings("ignore")

In [0]:
#Obtendo os dados.
url = "https://www.fundamentus.com.br/resultado.php"

#Isso irá nos trazer uma lista com todos os dataframes contidos no html, dessa forma vamos buscar apenas o primeiro dataframe.
df_teste = pd.read_html(url)

#tratar os dados antes de baixá-los, pois no site os decimais estão com vírgulas.
df = pd.read_html(url, decimal= ",", thousands=".")[0]



In [3]:
type(df_teste)

list

In [4]:
df

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
0,MNSA4,0.47,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"-208,15%","-362,66%",3.63,"-13,50%","145,70%",0.00,-9.105000e+06,-6.52,"-41,11%"
1,VNET3,0.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","12,99%",0.00,9.257250e+09,0.00,"-2,71%"
2,PORP4,2.40,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","-2,08%",0.00,2.239900e+07,0.00,"13,66%"
3,CFLU4,1000.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"8,88%","10,72%",1.10,"17,68%","32,15%",0.00,6.035100e+07,0.06,"8,14%"
4,CSTB3,150.00,0.00,0.00,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"40,85%","28,98%",2.60,"22,40%","20,11%",0.00,8.420670e+09,0.14,"31,91%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,UBBR11,14.75,1201.81,3.91,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.00,1.031720e+10,0.00,"10,58%"
882,UBBR3,18.00,1466.61,4.77,0.000,"0,00%",0.000,0.00,0.00,0.00,0.00,0.00,"0,00%","0,00%",0.00,"0,00%","0,33%",0.00,1.031720e+10,0.00,"10,58%"
883,CEPE6,19.30,1491.75,0.86,-17.684,"3,59%",0.161,-9.37,68.89,-0.32,307.50,254.51,"-25,67%","-1,19%",0.95,"0,27%","0,06%",537.14,1.667000e+09,3.36,"8,63%"
884,CEPE3,24.53,1895.99,1.10,-22.477,"0,00%",0.204,-11.91,87.55,-0.41,326.16,269.96,"-25,67%","-1,19%",0.95,"0,27%","0,06%",0.00,1.667000e+09,3.36,"8,63%"


In [0]:
for coluna in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
  df[coluna] = df[coluna].str.replace('.', '')                    #Retirar os pontos dos milhares.
  df[coluna] = df[coluna].str.replace(',', '.')                   #susbstituir a vírgula por ponto.
  df[coluna] = df[coluna].str.rstrip('%').astype('float') / 100   #Tirar o % e converter os valores em pontos float.

#Analisar os dados.


In [9]:
#Filtrar as empresas com liquides.
df1 = df[df["Liq.2meses"] > 1000000]
df1

,Papel,Cotação,P/L,P/VP,PSR,Div.Yield,P/Ativo,P/Cap.Giro,P/EBIT,P/Ativ Circ.Liq,EV/EBIT,EV/EBITDA,Mrg Ebit,Mrg. Líq.,Liq. Corr.,ROIC,ROE,Liq.2meses,Patrim. Líq,Dív.Brut/ Patrim.,Cresc. Rec.5a
17,SQIA3,21.77,-335.41,3.56,8.769,0.0000,2.477,4.50,-1759.28,7.50,-1437.65,86.55,-0.0050,-0.0261,7.63,-0.0034,-0.0106,1.949550e+07,4.314610e+08,0.20,0.2470
19,ANIM3,22.78,-263.88,3.67,2.143,0.0000,1.055,21.82,21.45,-2.02,30.45,14.91,0.0999,-0.0081,1.34,0.0540,-0.0139,2.227800e+07,6.904350e+08,1.80,0.0812
25,LPSB3,3.90,-134.39,3.22,3.901,0.0000,1.420,4.62,28.69,-16.39,21.40,10.47,0.1360,-0.0188,2.83,0.0825,-0.0240,4.895100e+06,1.785920e+08,0.05,-0.0914
26,BTOW3,73.25,-120.43,6.68,5.663,0.0000,2.387,8.32,248.93,-43.04,248.60,56.57,0.0227,-0.0470,1.95,0.0226,-0.0555,2.717220e+08,5.734430e+09,1.12,-0.0819
37,PRNR3,8.10,-50.44,4.15,0.884,0.0000,1.286,9.43,44.26,-8.31,50.22,13.11,0.0200,-0.0310,1.34,0.0376,-0.0822,1.720520e+06,7.432800e+07,1.16,0.2130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
868,OMGE3,29.99,187.75,1.81,5.025,0.0000,0.702,5.41,14.29,-1.63,22.75,14.92,0.3518,0.0322,3.81,0.0575,0.0096,2.077420e+07,2.816510e+09,1.42,0.6907
874,NTCO3,35.52,271.31,12.54,2.920,0.0000,1.991,22.06,26.20,-5.03,29.46,17.39,0.1115,0.0108,1.25,0.1165,0.0462,3.311400e+08,3.362340e+09,3.21,0.0000
876,VALE3,44.86,322.58,1.33,1.592,0.0315,0.539,5.45,3.85,-1.45,4.44,3.55,0.4137,-0.0102,1.72,0.1698,0.0041,1.624360e+09,1.788540e+11,0.55,0.1438
877,ALSO3,27.91,336.76,1.15,9.981,0.0000,0.680,5.69,17.83,-4.02,14.95,11.62,0.5597,0.0762,6.44,0.0430,0.0034,7.367620e+07,6.443630e+09,0.00,0.1860


In [16]:
#Filtrar as empresas com menor relação ev/ebit e maior roic.
ranking = pd.DataFrame()
ranking["pos"] = range (1,151)
ranking["EV/EBIT"] = df1[df1["EV/EBIT"]>0].sort_values(by=["EV/EBIT"])["Papel"][:150].values
ranking["ROIC"] = df1.sort_values(by=["ROIC"], ascending = False)["Papel"][:150].values
ranking

,pos,EV/EBIT,ROIC
0,1,PSSA3,WIZS3
1,2,SMLS3,PSSA3
2,3,SULA11,TOTS3
3,4,ENAT3,ODPV3
4,5,WIZS3,SMLS3
...,...,...,...
145,146,NTCO3,DTEX3
146,147,ANIM3,MDNE3
147,148,PNVL3,ROMI3
148,149,GNDI3,PRNR3


In [0]:
a = ranking.pivot_table(columns = "EV/EBIT", values="pos")
b = ranking.pivot_table(columns= "ROIC", values="pos")

In [20]:
t = pd.concat([a,b])
t

,AALR3,ABEV3,AGRO3,ALPA4,ALSO3,ALUP11,AMAR3,ANIM3,ARZZ3,B3SA3,BEEF3,BKBR3,BRDT3,BRFS3,BRKM5,BRML3,BRPR3,CAML3,CARD3,CCPR3,CCRO3,CEAB3,CESP3,CESP6,CIEL3,CMIG3,CMIG4,COCE5,COGN3,CPFE3,CPLE3,CPLE6,CRFB3,CSAN3,CSMG3,CSNA3,CVCB3,CYRE3,DIRR3,DTEX3,...,RLOG3,ROMI3,SAPR11,SAPR3,SAPR4,SBSP3,SEER3,SGPS3,SHUL4,SLCE3,SMLS3,SMTO3,SULA11,TASA4,TEND3,TGMA3,TIET11,TIET3,TIET4,TIMP3,TOTS3,TRIS3,TRPL4,TUPY3,UCAS3,UGPA3,UNIP6,USIM3,USIM5,VALE3,VIVA3,VIVT3,VIVT4,VLID3,VULC3,VVAR3,WEGE3,WIZS3,WSON33,YDUQ3
pos,100.0,72.0,96.0,145.0,102.0,8.0,24.0,147.0,138.0,141.0,42.0,110.0,94.0,80.0,107.0,81.0,125.0,114.0,88.0,101.0,77.0,65.0,123.0,121.0,39.0,16.0,15.0,75.0,135.0,60.0,29.0,31.0,63.0,103.0,41.0,35.0,7.0,112.0,26.0,142.0,...,45.0,130.0,11.0,12.0,9.0,33.0,47.0,111.0,79.0,66.0,2.0,113.0,3.0,98.0,20.0,36.0,86.0,95.0,85.0,54.0,83.0,49.0,37.0,23.0,91.0,105.0,25.0,50.0,44.0,6.0,136.0,92.0,90.0,32.0,40.0,126.0,NaN,5.0,21.0,82.0
pos,129.0,9.0,92.0,21.0,145.0,25.0,28.0,130.0,20.0,68.0,12.0,138.0,80.0,95.0,137.0,133.0,NaN,124.0,86.0,128.0,48.0,103.0,141.0,140.0,45.0,70.0,69.0,114.0,NaN,43.0,71.0,72.0,37.0,100.0,53.0,50.0,47.0,132.0,125.0,146.0,...,115.0,148.0,19.0,18.0,17.0,41.0,56.0,NaN,102.0,76.0,5.0,120.0,101.0,98.0,24.0,8.0,31.0,32.0,30.0,83.0,3.0,46.0,60.0,51.0,66.0,109.0,44.0,135.0,134.0,15.0,38.0,110.0,111.0,89.0,54.0,131.0,14.0,1.0,27.0,11.0


In [22]:
rank = t.dropna(axis=1).sum()
rank

AALR3     229.0
ABEV3      81.0
AGRO3     188.0
ALPA4     166.0
ALSO3     247.0
          ...  
VULC3      94.0
VVAR3     257.0
WIZS3       6.0
WSON33     48.0
YDUQ3      93.0
Length: 142, dtype: float64

In [33]:
#Esses multiplos não fazem muito sentido para empresas do setor financeiro.
pd.DataFrame(rank.sort_values(ascending=True))



,0
PSSA3,3.0
WIZS3,6.0
SMLS3,7.0
VALE3,21.0
SAPR4,26.0
SAPR3,30.0
SAPR11,30.0
ALUP11,33.0
LEVE3,38.0
MRFG3,40.0
